In [ ]:
import numpy as np
from utils_adaptive import run_adaptive
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams.update({'font.size': 14})
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('legend', fontsize=12)
matplotlib.rcParams['savefig.dpi'] = 1200

%load_ext autoreload
%autoreload 2
import time
import scipy.stats as scs
import math

from utils_carryover import theta1, theta2
from utils_adaptive import *
from utils_empirical import *

In [ ]:
out_dir = "../result/flu-adaptive/"
out_path = out_dir 
save_path = out_dir + "flu_adaptive-final.pkl"

In [ ]:
N = 50; 
tau = 1; fs_pct = 0.2; t0 = 3; 
num_mc = 1000

Y = import_MS_FLU_pos_data()
tau_ratio = -0.1

tau = np.mean(Y)*tau_ratio

summary = dict()
prec_thres = N/tau**2 * 4e-2

In [ ]:
start_time = time.time()
for T_max in [7, 14, 21, 28, 35]:
    
    np.random.seed(123)
    print(T_max)
    idx_N_list, idx_T_list, shuffle_list = sample_subblocks(Y, N, T_max, num_mc=num_mc)
    all_Ys = get_all_Ys(Y, idx_N_list, idx_T_list, shuffle_list)
    out = run_adaptive(tau, fs_pct=fs_pct, num_mc=num_mc,
                   t0=t0, adaptive=True, prec_thres=prec_thres, print_out=False, all_Ys=all_Ys)
    out_df = pd.DataFrame(out)
    print(np.mean(out_df["tau_adaptive"]**2), np.mean(out_df["tau_bm"]**2), np.mean(out_df["tau_oracle"]**2))

    summary[T_max] = out_df
    with open(save_path, 'wb') as handle:
        pickle.dump(summary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(time.time() - start_time)